In [1]:
import xmlrpc.client 
import pandas as pd

# Conexión con Odoo
username = "juan.cano@donsson.com"  # tu usuario
password = "1000285668"         # tu contraseña
url = "https://donsson.com"     # URL del servidor
db = "Donsson_produccion" # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")




fields_info = models.execute_kw(
    db, uid, password,
    'sale.order',  # Nombre técnico del modelo
    'fields_get',
    []
)

# Convertir a DataFrame para visualizar mejor
df_campos = pd.DataFrame.from_dict(fields_info, orient='index')
df_campos

,sortable,help,searchable,required,manual,readonly,depends,groups,company_dependent,translate,...,domain,context,states,relation_field,selection,digits,size,change_default,related,sanitize
origin,True,Reference of the document that generated this ...,True,False,False,False,[],False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
message_follower_ids,False,NaN,True,False,False,False,[],False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
create_date,True,Date on which sales order is created.,True,False,False,True,[],False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
store_id,True,Sucursal.,True,True,False,False,[],False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
categ_ids,True,NaN,True,False,False,False,[],False,False,NaN,...,"['|', ('section_id', '=', section_id), ('secti...",{'object_name': 'crm.lead'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
write_date,True,NaN,True,False,False,False,[],False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
website_order_line,True,Order Lines to be displayed on the website. Th...,True,False,False,True,[],False,False,NaN,...,NaN,NaN,NaN,order_id,NaN,NaN,NaN,NaN,NaN,NaN
website_message_ids,True,Website communication history,True,False,False,False,[],False,False,NaN,...,"[&, [model, =, sale.order], [type, =, comment]]",NaN,NaN,res_id,NaN,NaN,NaN,NaN,NaN,NaN
product_id,False,NaN,True,False,False,False,[],False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# TOTAL FACTURAS 

In [15]:
import xmlrpc.client 
from datetime import date
import pandas as pd

# Conexión con Odoo
username = "juan.cano@donsson.com"  # tu usuario
password = "1000285668"         # tu contraseña
url = "https://donsson.com"     # URL del servidor
db = "Donsson_produccion" # nombre de la base de datos


# --- Autenticación ---
common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# --- Fechas ---
from datetime import date, timedelta , datetime

weeks = 56

# --- Fechas automáticas ---
hoy = date.today()
fecha_fin = hoy.strftime("%Y-%m-%d")
fecha_inicio = (hoy - timedelta(weeks=weeks)).strftime("%Y-%m-%d")

# --- 1) Buscar facturas válidas ---

invoice_domain = [
    ("date_invoice", ">=", fecha_inicio),
    ("date_invoice", "<=", fecha_fin),
    ("type", "=", "out_invoice"),   # solo ventas
    ("state", "in", ["open", "paid"])
]

invoice_ids = models.execute_kw(
    db, uid, password,
    "account.invoice", "search",
    [invoice_domain]
)
print(f"Facturas encontradas: {len(invoice_ids)}")

# --- 2) Descargar las líneas de esas facturas ---
fields = ["product_id", "quantity", "price_subtotal", "invoice_id","create_date"] #account_analytic_id

records = []
limit = 20000
offset = 0

while True:
    result = models.execute_kw(
        db, uid, password,
        "account.invoice.line", "search_read",
        [[("invoice_id", "in", invoice_ids)]],
        {"fields": fields, "limit": limit, "offset": offset}
    )
    if not result:
        break
    records.extend(result)
    offset += limit
    print(f"Descargados {len(records)} registros...")

# --- 3) Pasar a DataFrame ---
line_df = pd.DataFrame(records).fillna(0)

# --- 3) Pasar a DataFrame ---
line_df = pd.DataFrame(records).fillna(0)

# Separar product_id
line_df["product_id_num"] = line_df["product_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
line_df["product_name"] = line_df["product_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Separar invoice_id
line_df["invoice_id_num"] = line_df["invoice_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
line_df["invoice_name"] = line_df["invoice_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

#df["company_id"] = df["company_id"].astype(str)

# Convertir fecha a datetime
line_df["date_invoice"] = pd.to_datetime(line_df["create_date"], errors="coerce")

# Eliminar las columnas originales problemáticas
line_df = line_df.drop(columns=["product_id", "invoice_id","create_date"])



invoice_fields = ["id", "store_id"]
invoices = models.execute_kw(
    db, uid, password,
    "account.invoice", "read",
    [invoice_ids],
    {"fields": invoice_fields}
)
invoice_df = pd.DataFrame(invoices)

# Separar store_id
invoice_df["store_id_num"] = invoice_df["store_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
invoice_df["store_name"] = invoice_df["store_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)



df_merged = line_df.merge(invoice_df[["id", "store_name","store_id_num"]], left_on="invoice_id_num", right_on="id", how="left")
df_merged = df_merged.drop(columns=["id_x","id_y","invoice_id_num","product_id_num"])




df_merged.to_parquet(f"ventashistoricas{weeks}semanas.parquet", index=False)

print(f"Total de líneas descargadas lines account: {len(line_df)}")


#para 52 semanas se demora aproximadamente 6 minutos
#para 56 semanas se demora aproximadamente 7 minutos

Facturas encontradas: 54838
Descargados 20000 registros...
Descargados 40000 registros...
Descargados 60000 registros...
Descargados 80000 registros...
Descargados 100000 registros...
Descargados 120000 registros...
Descargados 140000 registros...
Descargados 160000 registros...
Descargados 180000 registros...
Descargados 200000 registros...
Descargados 213281 registros...
Total de líneas descargadas lines account: 213281


In [16]:
print(f"Fecha primera factura",df_merged["date_invoice"].min())
print(f"Fecha ultima factura",df_merged["date_invoice"].max())

Fecha primera factura 2024-08-23 12:58:15
Fecha ultima factura 2025-09-19 13:51:58


In [18]:
df_merged.sample(10)

,price_subtotal,quantity,product_name,invoice_name,date_invoice,store_name,store_id_num
63137,339200.0,4.0,[DAB08154025] DA8154 FILTRO DE AIRE - HYSTER -...,PV2E40289 Mostrador Calle 6 T2/20369,2025-05-29 20:36:38,SUCURSAL CALLE 6,2
43917,98700.0,3.0,[DCS00342189] GS342 FILTRO COMBUSTIBLE SEPARAD...,FMDE3191,2025-07-07 13:29:54,SUCURSAL MEDELLIN,8
12054,470400.0,12.0,[BCS00252125] GS252 FILTRO COMBUSTIBLE BOBCAT ...,FCOT25977,2025-08-30 14:44:31,PRINCIPAL COTA,1
181657,20400.0,1.0,"[BCE00046125] G046 FILTRO COMBUSTIBLE PERKINS,...",PV2E34301 Mostrador Calle 6 T2/18927,2024-10-23 14:27:31,SUCURSAL CALLE 6,2
147367,48900.0,1.0,[DAB08108025] DA8108 FILTRO AIRE- HINO DUTRO 3...,PV3E16429 Mostrador Valladolid T2/13234,2024-12-27 15:53:02,SUCURSAL VALLADOLID,6
16346,124600.0,2.0,"[DAB04881025] DA4881 FILTRO AIRE 2_ CASE, JOHN...",FMDE3665,2025-08-23 16:38:37,SUCURSAL MEDELLIN,8
25647,138800.0,4.0,[DAB04968025] DA4968 FILTRO AIRE 2_ HINO FC (0...,FCAL5167,2025-08-06 15:11:14,SUCURSAL CALI,15
122692,1165500.0,7.0,[DAB02728025] DA2728 FILTRO AIRE 1_ TEREX. (02...,FCOT23016 OC:4502497610,2025-02-13 18:08:15,PRINCIPAL COTA,1
95524,49500.0,1.0,[DCE10126137] G126A10 FILTRO separador agua RA...,PV6E16638 Mostrador Barranquilla T2/22427,2025-04-01 21:46:42,SUCURSAL BARRANQUILLA,10
114975,99600.0,2.0,[BCS00025125] GS025 FILTRO COMBUSTIBLE CUMMINS...,FMDE1915,2025-02-26 17:13:03,SUCURSAL MEDELLIN,8


In [19]:
df_merged.groupby("store_id_num")["store_name"].count()

store_id_num
1     50888
2     34068
6     17134
7     13019
8     26691
10    37843
11    19467
15    14171
Name: store_name, dtype: int64

In [21]:
df_merged[df_merged["store_id_num"]==11]

,price_subtotal,quantity,product_name,invoice_name,date_invoice,store_name,store_id_num
8,97500.0,1.0,[DAB02957025] DA2957 FILTRO AIRE DONSSON - FOR...,PV7E8224 Mostrador Bucaramanga T1/7980,2025-09-19 13:43:35,SUCURSAL BUCARAMANGA,11
123,31500.0,1.0,[DAB02570025] DA2570 FILTRO AIRE DONSSON - PER...,FBUC6554,2025-09-18 21:30:11,SUCURSAL BUCARAMANGA,11
124,25500.0,1.0,[DAB14570025] DA4570A FILTRO AIRE 2_ DONSSON -...,FBUC6554,2025-09-18 21:30:11,SUCURSAL BUCARAMANGA,11
128,184500.0,3.0,[DAB04887025] DA4887 FILTRO AIRE SEGURIDAD REN...,PV7E8223 Mostrador Bucaramanga T2/2524,2025-09-18 21:23:27,SUCURSAL BUCARAMANGA,11
130,93300.0,1.0,[BCS00312125] GS312 FILTRO COMBUSTIBLE KENWORT...,PV7E8222 Mostrador Bucaramanga T2/2523,2025-09-18 21:18:52,SUCURSAL BUCARAMANGA,11
...,...,...,...,...,...,...,...
213276,135000.0,2.0,[BCS00363125] GS363 FILTRO COMBUSTIBLE 2° MACK...,FBUC5007,2024-08-23 12:58:15,SUCURSAL BUCARAMANGA,11
213277,114800.0,1.0,[BHS00325125] GS325 FILTRO HIDRAULICO TRANSMIS...,FBUC5007,2024-08-23 12:58:15,SUCURSAL BUCARAMANGA,11
213278,106200.0,3.0,[BHS00504125] GS504 FILTRO HIDR. TRANSMIS. FOR...,FBUC5007,2024-08-23 12:58:15,SUCURSAL BUCARAMANGA,11
213279,227400.0,3.0,[BLS10077125] GS077A FILTRO ACEITE BY PASS VOL...,FBUC5007,2024-08-23 12:58:15,SUCURSAL BUCARAMANGA,11


# FACTURAS ULTIMAS 12 SEMANAS

In [ ]:
import xmlrpc.client 
from datetime import date
import pandas as pd

# Conexión con Odoo
username = "juan.cano@donsson.com"  # tu usuario
password = "1000285668"         # tu contraseña
url = "https://donsson.com"     # URL del servidor
db = "Donsson_produccion" # nombre de la base de datos


# --- Autenticación ---
common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# --- Fechas ---
from datetime import date, timedelta

# --- Fechas automáticas ---
hoy = date.today()
fecha_fin = hoy.strftime("%Y-%m-%d")
fecha_inicio = (hoy - timedelta(weeks=12)).strftime("%Y-%m-%d")
# --- 1) Buscar facturas válidas ---

invoice_domain = [
    ("date_invoice", ">=", fecha_inicio),
    ("date_invoice", "<=", fecha_fin),
    ("type", "=", "out_invoice"),   # solo ventas
    ("state", "in", ["open", "paid"])
]

invoice_ids = models.execute_kw(
    db, uid, password,
    "account.invoice", "search",
    [invoice_domain]
)
print(f"Facturas encontradas: {len(invoice_ids)}")



# --- 2) Descargar las líneas de esas facturas ---
fields = ["product_id", "quantity", "price_subtotal", "invoice_id","create_date","company_id","store"] #company id no va siempre muestra donsson

records = []
limit = 20000
offset = 0

while True:
    result = models.execute_kw(
        db, uid, password,
        "account.invoice.line", "search_read",
        [[("invoice_id", "in", invoice_ids)]],
        {"fields": fields, "limit": limit, "offset": offset}
    )
    if not result:
        break
    records.extend(result)
    offset += limit
    print(f"Descargados {len(records)} registros...")

# --- 3) Pasar a DataFrame ---
lines_df = pd.DataFrame(records).fillna(0)

# --- 3) Pasar a DataFrame ---
lines_df = pd.DataFrame(records).fillna(0)

# Separar product_id
lines_df["product_id_num"] = lines_df["product_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
lines_df["product_name"] = lines_df["product_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Separar invoice_id
lines_df["invoice_id_num"] = lines_df["invoice_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
lines_df["invoice_name"] = lines_df["invoice_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

lines_df["company_id"] = lines_df["company_id"].astype(str)

# Convertir fecha a datetime
lines_df["date_invoice"] = pd.to_datetime(df["create_date"], errors="coerce")

# Eliminar las columnas originales problemáticas
lines_df = lines_df.drop(columns=["product_id", "invoice_id","create_date"])


invoice_fields = ["id", "store_id"]
invoices = models.execute_kw(
    db, uid, password,
    "account.invoice", "read",
    [invoice_ids],
    {"fields": invoice_fields}
)
invoice_df = pd.DataFrame(invoices)

df_merged = lines_df.merge(invoice_df[["id", "store_id"]], left_on="invoice_id", right_on="id", how="left")
df_merged.drop(columns=["id"], inplace=True)



df_merged.to_parquet("historico3mesesEMA.parquet", index=False)

print(f"Total de líneas descargadas: {len(df)}")

Facturas encontradas: 12171
Descargados 20000 registros...
Descargados 40000 registros...
Descargados 47007 registros...
Total de líneas descargadas: 47007


# COSTO DEL PRODUCTO

In [3]:
import xmlrpc.client
import pandas as pd


# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

campos = models.execute_kw(
    db , uid , password,
    "cs.analisis.costo","fields_get",
    [],

)

df_campos = pd.DataFrame(campos)
#df_campos.info()


productos_costo = models.execute_kw(
    db , uid , password,
    "cs.analisis.costo","search_read",
    [[]],
    {"fields":["producto_costo_unitario","producto_id"]}


)

# Convertir en df
df_productos_costo = pd.DataFrame(productos_costo)


# Separar lista anidada
df_productos_costo[["product_id_num","product_name"]] = (df_productos_costo["producto_id"].apply(pd.Series))

#Agregar columna estandarizada
df_productos_costo["product_name"] = df_productos_costo["product_name"].str.strip()

#DF normalizado
df_productos_costo = df_productos_costo[["product_name","producto_costo_unitario"]]

df_productos_costo.to_parquet("costo_productos.parquet")


# ANALISIS COSTOS

In [14]:
import xmlrpc.client
import pandas as pd


# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

productos_ema = models.execute_kw(
    db, uid, password,
    "cs.analisis.costo", "search_read",[],
    #[
    #    [["semana","=",36]]
    #],
    {"fields": ["sucursal_id","almacen_id","product_name","ano","semana","ema","producto_costo_unitario" ]}
)

df_campos = pd.DataFrame(productos_ema)
df_campos.to_csv("real_ema.csv")

#timpo aprox  1 minutos

# VENTAS PERDIDAS

In [ ]:
import xmlrpc.client 
from datetime import date
import pandas as pd

# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# ===============================
# Leer ventas perdidas desde 2025
# ===============================

ventas_perdidas_ids = models.execute_kw(
    db, uid, password,
    "ventas.perdidas", "search",
    [[["fecha", ">=", "2024-12-01"]], ["almacenamiento_tipo","=","agotado"]] 
)

ventas_perdidas = models.execute_kw(
    db, uid, password,
    "ventas.perdidas", "read",
    [ventas_perdidas_ids],
    {"fields": ["origen", "store_id", "warehouse_id", "cliente_id",
                "fecha", "product_ref", "product_id",
                "cantidad", "cantidad_existencia","cantidad_reservada","almacenamiento_tipo"]}
)

df = pd.DataFrame(ventas_perdidas)

# ===============================
# Normalizar columnas con tuplas (many2one)
# ===============================
def extract_id(val):
    if isinstance(val, (list, tuple)) and len(val) > 0:
        return val[0]
    return None

def extract_name(val):
    if isinstance(val, (list, tuple)) and len(val) > 1:
        return val[1]
    return str(val)

df["product_id_num"] = df["product_id"].apply(extract_id)
df["product_name"]   = df["product_id"].apply(extract_name)

df["cliente_id_num"] = df["cliente_id"].apply(extract_id)
df["cliente_name"]   = df["cliente_id"].apply(extract_name)

df["store_id_num"] = df["store_id"].apply(extract_id)
df["store_name"]   = df["store_id"].apply(extract_name)

df["warehouse_id_num"] = df["warehouse_id"].apply(extract_id)
df["warehouse_name"]   = df["warehouse_id"].apply(extract_name)

df = df.drop(["product_id_num","store_id","warehouse_id","cliente_id","id","cliente_id_num","warehouse_id_num","warehouse_name"],axis=1) #eliminar columnas que no sirven

# ===============================
# Forzar tipos seguros
# ===============================
for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].astype("string")

# ===============================
# Exportar a Parquet
# ===============================
try:
    df.to_parquet("ventas_perdidas_2025.parquet", index=False, engine="pyarrow", compression="snappy")
    print("✅ Exportación exitosa con pyarrow")
except Exception as e:
    print("⚠️ pyarrow falló, probando con fastparquet:", e)
    df.to_parquet("ventas_perdidas_2025.parquet", index=False, engine="fastparquet", compression="snappy")
    print("✅ Exportación exitosa con fastparquet")
    

    ##TARDA MAS O MENOS 5 MINUTOS TODO 2025


✅ Exportación exitosa con pyarrow


# MOTOR BUSQUEDA

In [3]:
import xmlrpc.client
import pandas as pd

# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# ===============================
# Ver campos disponible
# ===============================
fields = models.execute_kw(
    db, uid, password,
    "mrp.routing.workcenter", "fields_get",
    []
)

for field, info in sorted(fields.items(), key=lambda x: x[1].get("string", "")):
    print(f"{field:25} | {info.get('type')} | {info.get('string')}")



total_fuerza_lab          | float | Cantidad fuerza laboral
company_id                | many2one | Company
create_uid                | many2one | Created by
create_date               | datetime | Created on
note                      | text | Description
display_name              | char | Display Name
efficiency_time           | float | Eficiencia
id                        | integer | Id
__last_update             | datetime | Last Modified on
write_uid                 | many2one | Last Updated by
write_date                | datetime | Last Updated on
name                      | char | Name
cycle_nbr                 | float | Number of Cycles
hour_nbr                  | float | Number of Hours
routing_id                | many2one | Parent Routing
sequence                  | integer | Sequence
total_nbr_minimo          | float | Tiempo en segundos optimo
valor_promedio_segundo    | float | Valor segundos promedio
workcenter_id             | many2one | Work Center


In [15]:
import xmlrpc.client
import pandas as pd

# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# 1) Traer nóminas (slips) solo de 2025
# ========================================
slips = models.execute_kw(
    db, uid, password,
    "hr.payslip", "search_read",
    [[["date_from", ">=", "2024-01-01"], ["date_to", "<=", "2025-12-31"]]],
    {"fields": ["id", "employee_id", "name", "date_from", "date_to"]}
)
df_slips = pd.DataFrame(slips)


# ========================================
# 2) Traer líneas de nómina (solo de los slips obtenidos)
# ========================================
slip_ids = df_slips["id"].tolist()
lines = models.execute_kw(
    db, uid, password,
    "hr.payslip.line", "search_read",
    [[["slip_id", "in", slip_ids]]],
    {"fields": ["id", "slip_id", "employee_id", "salary_rule_id", "total"]}
)
df_lines = pd.DataFrame(lines)

# ===============================
# 3) Expandir campos Many2one (id, name)
# ===============================
def extract_name(val):
    if isinstance(val, list) and len(val) == 2:
        return val[1]
    return None

def extract_id(val):
    if isinstance(val, list) and len(val) == 2:
        return val[0]
    return None

df_lines["employee_name"] = df_lines["employee_id"].apply(extract_name)
df_lines["slip_name"]     = df_lines["slip_id"].apply(extract_name)
df_lines["rule_name"]     = df_lines["salary_rule_id"].apply(extract_name)

# 🔑 Extraer ID de slip_id para poder hacer el merge
df_lines["slip_id_only"] = df_lines["slip_id"].apply(extract_id)

# ===============================
# 4) Unir líneas con fechas de slip
# ===============================
df_slips["employee_name"] = df_slips["employee_id"].apply(extract_name)

df = pd.merge(
    df_lines,
    df_slips[["id", "employee_name", "date_from", "date_to"]],
    left_on="slip_id_only",
    right_on="id",
    how="left",
    suffixes=("", "_slip")
)

# ===============================
# 5) Filtrar reglas relevantes
# ===============================
df_filtro = df[df["rule_name"].isin(
    ["Salario Básico", "Salario Mínimo", "Salario Neto"]
)].copy()

# ===============================
# 6) Agregar columna período (Año-Mes)
# ===============================
df_filtro["periodo"] = pd.to_datetime(df_filtro["date_from"]).dt.to_period("M")

# ===============================
# 7) Pivotear para resumir salarios
# ===============================
tabla = df_filtro.pivot_table(
    index=["employee_name", "periodo"],
    columns="rule_name",
    values="total",
    aggfunc="sum",
    fill_value=0
).reset_index()

# ===============================
# 8) Renombrar columnas
# ===============================
tabla = tabla.rename(columns={
    "Salario Básico": "salario_basico",
    "Salario Mínimo": "salario_minimo",
    "Salario Neto": "salario_neto"
})

# ===============================
# 9) Calcular salario base = máximo entre básico y mínimo
# ===============================
tabla["salario_base"] = tabla[["salario_basico", "salario_minimo"]].max(axis=1)

# ===============================
# 10) Tabla final
# ===============================
resultado = tabla[["employee_name", "periodo", "salario_base", "salario_neto"]]

print("✅ Resumen mensual de salarios")

#aprox2min

✅ Resumen mensual de salarios


In [18]:
r =resultado.sort_values(["salario_neto"],ascending=False)
r = r[r["periodo"]=="2025-09"]
r.head(30)

rule_name,employee_name,periodo,salario_base,salario_neto
927,GERMAN RICARDO BETANCOURT GUTIERREZ,2025-09,1423500.0,15551900.0
1364,JUAN IGNACIO NIETO NEIVA,2025-09,3600000.0,13414292.0
1023,JAIME ALBERTO CAÑON RAMIREZ,2025-09,1423500.0,7492260.0
906,GERMAN ANTONIO MORALES CARDENAS,2025-09,1423500.0,6936680.0
1266,JUAN CARLOS BETANCOURT GUTIERREZ,2025-09,4400000.0,6238869.0
1870,OMAR YADIR HERNANDEZ PULGARIN,2025-09,1423500.0,5736458.0
1485,LEONARDO MORA CUBILLOS,2025-09,1423500.0,5316042.0
1131,JESUS ADONAY GONZALEZ FLOREZ,2025-09,1450000.0,5081168.0
2068,SIMON BETANCOURT DEL PINO,2025-09,5350000.0,4921800.0
960,HUGO ALEJANDRO ZUBIETA FERNANDEZ,2025-09,1423500.0,4903609.0
